In [1]:
#Import library os, requests, threading, urlib, dan time
import os
import requests
import threading
import urllib.request, urllib.error, urllib.parse
import time

#URL yang digunakan untuk mendownload file
url = "https://apod.nasa.gov/apod/image/1901/LOmbradellaTerraFinazzi.jpg"

#Memecah file untuk di download
def buildRange(value, numsplits):
    lst = []
    for i in range(numsplits):
        if i == 0:
            lst.append('%s-%s' % (i, int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))))
        else:
            lst.append('%s-%s' % (int(round(1 + i * value/(numsplits*1.0),0)), int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))))
    return lst

#Kelas split buffer threads
class SplitBufferThreads(threading.Thread):
    #Inisialisasi split buffer thread, url, byte range, dan request
    def __init__(self, url, byteRange):
        super(SplitBufferThreads, self).__init__()
        self.__url = url
        self.__byteRange = byteRange
        self.req = None
    
    #Meminta request dari file header
    def run(self):
        self.req = urllib.request.Request(self.__url,  headers={'Range': 'bytes=%s' % self.__byteRange})
    
    #Membuka file yang didapat dari hasil request
    def getFileData(self):
        return urllib.request.urlopen(self.req).read()

#Membuat fungsi utama
def main(url=None, splitBy=3):
    #Jumlah waktu mendownload file
    start_time = time.time()
    
    #Memasukkan url secara manual jika url tidak bisa dibuka
    if not url:
        print("Please Enter some url to begin download.")
        return
    
    #Memberikan nama file berupa kata terakhir dari url
    fileName = url.split('/')[-1]
    
    #Mendefinisikan tipe dan ukuran file yang didownload
    sizeInBytes = requests.head(url, headers={'Accept-Encoding': 'identity'}).headers.get('content-length', None)
    
    #Menampilkan ukuran file yang didownload dalam bytes
    print("%s bytes to download." % sizeInBytes)
    
    #Menampilkan pesan jika ukuran tidak dapat ditentukan
    if not sizeInBytes:
        print("Size cannot be determined.")
        return

    #Array untuk menampung hasil data
    dataLst = []
    
    #Melakukan pembagian thread
    for idx in range(splitBy):
        byteRange = buildRange(int(sizeInBytes), splitBy)[idx]
        bufTh = SplitBufferThreads(url, byteRange)
        bufTh.start()
        bufTh.join()
        dataLst.append(bufTh.getFileData())

    content = b''.join(dataLst)

    #Memeriksa array yang berisi hasil data
    if dataLst:
        #Menggantikan file jika nama file sudah ada
        if os.path.exists(fileName):
            os.remove(fileName)
        #Menampilkan pesan total waktu download
        print("--- %s seconds ---" % str(time.time() - start_time))
        #Membuka file
        with open(fileName, 'wb') as fh:
            #Menulis konten ke file
            fh.write(content)
        #Menampilkan pesan file telah ditulis
        print("Finished Writing file %s" % fileName)

#Memanggil fungsi utama
if __name__ == '__main__':
    main(url)

3670260 bytes to download.
--- 31.780571937561035 seconds ---
Finished Writing file LOmbradellaTerraFinazzi.jpg
